In [43]:
from sklearn.preprocessing import MinMaxScaler

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import requests

In [2]:
import tensorflow

2024-08-26 14:04:15.627809: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-26 14:04:15.628391: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-26 14:04:15.630673: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-26 14:04:15.636092: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-26 14:04:15.645174: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [3]:
from tensorflow.keras import models
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from tensorflow.keras.layers import Dense, Reshape
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Normalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [40]:
df

,datetime,1. open,2. high,3. low,4. close,5. volume
0,2010-01-04 04:15:00,6.411,6.418,6.399,6.409,2800
1,2010-01-04 04:35:00,6.426,6.433,6.414,6.424,2800
2,2010-01-04 04:45:00,6.426,6.433,6.414,6.424,2800
3,2010-01-04 05:05:00,6.432,6.438,6.419,6.429,2800
4,2010-01-04 05:10:00,6.432,6.439,6.419,6.430,2800
...,...,...,...,...,...,...
547950,2023-11-30 19:35:00,189.868,189.920,189.858,189.889,385
547951,2023-11-30 19:40:00,189.888,190.000,189.878,189.959,5912
547952,2023-11-30 19:45:00,189.958,190.000,189.888,189.949,10184
547953,2023-11-30 19:50:00,189.948,190.000,189.898,189.959,5504


In [39]:
df.isna().sum()

datetime     0
1. open      0
2. high      0
3. low       0
4. close     0
5. volume    0
dtype: int64

In [4]:
df = pd.read_csv("raw_data/apple_stock.csv")

In [45]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

In [62]:
mm_scaler = MinMaxScaler().set_output(transform="pandas")

In [49]:
imputer = FunctionTransformer(impute_missing)

In [50]:
converter = FunctionTransformer(convert_time_sin_cos)

In [51]:
preproc_pipeline = Pipeline([("imputer", imputer),("time_converter", converter),("scaler", mm_scaler)])

In [52]:
preproc_pipeline

Pipeline(steps=[('imputer',
                 FunctionTransformer(func=<function impute_missing at 0x7fbee56267a0>)),
                ('time_converter',
                 FunctionTransformer(func=<function convert_time_sin_cos at 0x7fbee5626050>)),
                ('scaler', MinMaxScaler())])

In [42]:
def impute_missing(df):
    data = df.copy()
    data_imputed = data.fillna(method = "ffill", axis=1)
    return data_imputed

In [36]:
def convert_time_sin_cos(df):
    data = df.copy()
    data["datetime"] = pd.to_datetime(data["datetime"])
    data.drop(columns = "1. open", inplace=True)
    data.rename(columns = {"4. close": "target"}, inplace=True)
    data['day_of_year'] = data['datetime'].dt.dayofyear
    data['time_of_day'] = data['datetime'].dt.hour * 3600 + data['datetime'].dt.minute * 60 + data['datetime'].dt.second
    data['day_of_year_norm'] = data['day_of_year'] / 365.0
    data['time_of_day_norm'] = data['time_of_day'] / 86400.0
    data['cos_time_of_day'] = np.cos(2 * np.pi * data['time_of_day_norm'])
    data['sin_time_of_day'] = np.sin(2 * np.pi * data['time_of_day_norm'])
    data["cos_day_of_year"] = np.cos(2 * np.pi * data['day_of_year_norm'])
    data["sin_day_of_year"] = np.sin(2 * np.pi * data['day_of_year_norm'])
    df_converted = data.drop(columns = ["day_of_year", "time_of_day", "day_of_year_norm", "time_of_day_norm", "datetime"])
    return df_converted

In [5]:
df["datetime"] = pd.to_datetime(df["datetime"])

In [6]:
df_test = df.drop(columns = "1. open", inplace=True)

In [7]:
df_test.rename(columns = {"4. close": "target"}, inplace=True)

In [8]:
df_test['day_of_year'] = df_test['datetime'].dt.dayofyear
df_test['time_of_day'] = df_test['datetime'].dt.hour * 3600 + df_test['datetime'].dt.minute * 60 + df_test['datetime'].dt.second

In [9]:
df_test['day_of_year_norm'] = df_test['day_of_year'] / 365.0
df_test['time_of_day_norm'] = df_test['time_of_day'] / 86400.0

In [10]:
df_test['cos_time_of_day'] = np.cos(2 * np.pi * df_test['time_of_day_norm'])
df_test['sin_time_of_day'] = np.sin(2 * np.pi * df_test['time_of_day_norm'])

In [11]:
df_test["cos_day_of_year"] = np.cos(2 * np.pi * df_test['day_of_year_norm'])
df_test["sin_day_of_year"] = np.sin(2 * np.pi * df_test['day_of_year_norm'])

In [12]:
df_test = df_test.drop(columns = ["day_of_year", "time_of_day", "day_of_year_norm", "time_of_day_norm", "datetime"])

In [13]:
df_test

,2. high,3. low,target,5. volume,cos_time_of_day,sin_time_of_day,cos_day_of_year,sin_day_of_year
0,6.418,6.399,6.409,2800,0.442289,0.896873,0.997630,0.068802
1,6.433,6.414,6.424,2800,0.362438,0.932008,0.997630,0.068802
2,6.433,6.414,6.424,2800,0.321439,0.946930,0.997630,0.068802
3,6.438,6.419,6.429,2800,0.237686,0.971342,0.997630,0.068802
4,6.439,6.419,6.430,2800,0.216440,0.976296,0.997630,0.068802
...,...,...,...,...,...,...,...,...
547950,189.920,189.858,189.889,385,0.402747,-0.915311,0.860961,-0.508671
547951,190.000,189.878,189.959,5912,0.422618,-0.906308,0.860961,-0.508671
547952,190.000,189.888,189.949,10184,0.442289,-0.896873,0.860961,-0.508671
547953,190.000,189.898,189.959,5504,0.461749,-0.887011,0.860961,-0.508671


In [14]:
train_size = 0.8
index = round(train_size*df_test.shape[0])

df_train = df_test.iloc[:index]
df_test_2 = df_test.iloc[index:]

In [63]:
df_train = mm_scaler.fit_transform(df_train)

In [64]:
df_train

,x0,x1,x2,x3,x4,x5,x6,x7
0,0.004822,0.004820,0.004816,0.000012,0.733643,0.948436,0.999111,0.534402
1,0.004932,0.004930,0.004926,0.000012,0.693026,0.966004,0.999111,0.534402
2,0.004932,0.004930,0.004926,0.000012,0.672172,0.973465,0.999111,0.534402
3,0.004969,0.004967,0.004963,0.000012,0.629569,0.985671,0.999111,0.534402
4,0.004976,0.004967,0.004970,0.000012,0.618762,0.988148,0.999111,0.534402
...,...,...,...,...,...,...,...,...
438359,0.872850,0.872716,0.872320,0.001801,0.079661,0.231350,0.040351,0.696797
438360,0.872668,0.872680,0.872261,0.001379,0.085726,0.222215,0.040351,0.696797
438361,0.872382,0.872460,0.872518,0.001482,0.091991,0.213212,0.040351,0.696797
438362,0.873158,0.872892,0.873367,0.002043,0.098455,0.204345,0.040351,0.696797


In [65]:
y_test = df_test_2[["target"]]
X_test = df_test_2.drop(columns = "target")

In [67]:
y_train = df_train[["x3"]]
X_train = df_train.drop(columns = "x3")

In [68]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [69]:
X_train.shape

(438364, 7, 1)

In [70]:
reg_l2 = regularizers.L2(0.01)

In [71]:
es = EarlyStopping(patience=10, restore_best_weights=True)

In [72]:
model = Sequential()
model.add(layers.LSTM(units = 128, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(units = 64, return_sequences = True, input_shape = (X_train.shape[1], 1), kernel_regularizer=reg_l2))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(units = 64, return_sequences = True, input_shape = (X_train.shape[1], 1),kernel_regularizer=reg_l2))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(units = 32, return_sequences = True, input_shape = (X_train.shape[1], 1), kernel_regularizer=reg_l2))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(units = 8, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(layers.Dense(1, activation="linear"))

/home/yurko/.pyenv/versions/3.10.6/envs/portfolio_manager/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [73]:
model.compile(loss="mse", optimizer="adam", metrics=["mae"])

In [75]:
history = model.fit(X_train, y_train, epochs=30, callbacks=[es], batch_size=64)